# Clustering Neighbourhoods in Toronto

## Part 1: Scrape neighbourhoods data into pandas dataframe and pre-process dataset

#### Setup Libraries & Packages

In [12]:
pip install lxml html5lib beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd

#### Scrape neighbourhoods data

In [39]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url)
df = dfs[0]

#### Remove records where a borough is not assigned

In [40]:
df.drop(df[df['Borough']=="Not assigned"].index, inplace = True) 

In [43]:
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Identify the number of rows and columns in cleaned dataframe

In [50]:
print('The dataframe has ', df.shape[0], ' rows and ', df.shape[1], ' columns.')

The dataframe has  103  rows and  3  columns.
